# Datapath example 4

This notebook provides some miscellaneaous examples for using the derivapy
`datapath` module and `PathBuilder` interface. It assumes that you understand 
the concepts presented in the example 3 notebook.

You should also read the ERMrest documentation and the derivapy wiki. There are
more advanced concepts in this notebook that are demonstrated but not fully
(re)explained here, as the concepts are explained in other documentation.

## Exampe Data Model
The examples require that you understand a little bit about the example
catalog data model, which is based on the FaceBase project.

### Key tables
- `'dataset'` : represents a unit of data usually a `'study'` or `'experiment'`
- `'sample'` : a biosample
- `'assay'` : a bioassay (typically RNA-seq or ChIP-seq assays)

### Relationships
- `dataset <- sample`: A dataset may have one to many samples. I.e., there 
  is a foreign key reference from sample to dataset.
- `sample <- assay`: A sample may have one to many assays. I.e., there is a
  foreign key reference from assay to sample.

In [1]:
# Import deriva modules
from deriva_common import ErmrestCatalog, get_credential

In [2]:
# Connect with the deriva catalog
protocol = 'https'
hostname = 'www.facebase.org'
catalog_number = 1
credential = None
# If you need to authenticate, use Deriva Auth agent and get the credential
# credential = get_credential(hostname)
catalog = ErmrestCatalog(protocol, hostname, catalog_number, credential)

In [3]:
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()

# Get some local variable handles to tables for convenience
dataset = pb.isa.dataset
sample = pb.isa.sample
assay = pb.isa.assay

## Attributes (a.k.a., Projection) Examples
When we fetch specific named attributes, this is known in database terms as a "projection" (or "project" if used as a verb).

### Example: projecting all column of a table
Passing a table object to the `attributes(...)` method will project all (i.e., `*`) of its attributes.

In [4]:
path = dataset.attributes(dataset)
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/isa:dataset/*


In [5]:
path.entities(limit=1).dataframe

,_keywords,accession,description,funding,gene_summary,human_anatomic,id,mouse_genetic,project,release_date,show_in_jbrowse,status,study_design,summary,thumbnail,title,view_gene_summary,view_related_datasets
0,Epigenetic landscapes and regulatory divergenc...,FB00000827,Tg(hg19_chr7:145843942-145844366::LacZ) activi...,U01 DE024430,None,None,14062,None,307,2016-01-01,None,3,None,Activity of human neural crest enhancer in E11...,442,Activity of human neural crest enhancer in E11...,None,None


### Example: project all columns of an aliased table
Similarly, pass a table alias to the `attributes(...)` method works the same.

In [6]:
D = dataset.as_('D')
path = D.attributes(dataset)
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/*


In [7]:
path.entities(limit=1).dataframe

,_keywords,accession,description,funding,gene_summary,human_anatomic,id,mouse_genetic,project,release_date,show_in_jbrowse,status,study_design,summary,thumbnail,title,view_gene_summary,view_related_datasets
0,Epigenetic landscapes and regulatory divergenc...,FB00000827,Tg(hg19_chr7:145843942-145844366::LacZ) activi...,U01 DE024430,None,None,14062,None,307,2016-01-01,None,3,None,Activity of human neural crest enhancer in E11...,442,Activity of human neural crest enhancer in E11...,None,None


### Example: renaming columns in an entity set
As we get into more complete data paths and attribute projections from those data paths, we will run into more sitations where the column names from linked entity sets that we want to include in our output may collide. Normally, this would result in an error and the catalog would not be able to return results. Later we will make use of this feature, but just to introduce it we will show a trivial example.

In [8]:
path = D.attributes(accession_number=D.accession, date_of_release=D.release_date)
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/accession_number:=D:accession,date_of_release:=D:release_date


In [9]:
path.entities(limit=1).dataframe

,accession_number,date_of_release
0,FB00000827,2016-01-01


## Filtering Examples

### Example: using logical conjunction in filters
This example shows how to combine two comparisons with a conjuncting (i.e., `and` operator). Because the python "`and`" keyword cannot be overloaded, we instead overload the python "`&`" bitwise-and operator. This approach has become customary among many similar data access libraries.

In [10]:
path = dataset.link(sample).link(assay).filter(
    ((assay.sample_type == 'ChIP-seq') & (assay.selection == 'H3K27AC')))

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/isa:sample/isa:assay/(sample_type=ChIP-seq)&(selection=H3K27AC)


In [11]:
path.entities().dataframe

,alignment_id,cell_count,dataset,fragmentation_method,id,isolation_protocol,library_id,markers,molecule_type,pretreatment,...,reagent_batch_number,reagent_catalog_number,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,tracks_id
0,45,NA,14068,Sonication,5,,65,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,1,medial nasal process,excision,ChIP-seq,H3K27AC,25
1,52,NA,14068,Sonication,12,,72,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,4,mandibular process,excision,ChIP-seq,H3K27AC,32
2,54,NA,14068,Sonication,14,,74,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,3,latero nasal process,excision,ChIP-seq,H3K27AC,34
3,59,NA,14068,Sonication,19,,79,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,2,maxillary process,excision,ChIP-seq,H3K27AC,39
4,61,NA,14130,Sonication,21,None,81,Histology,genomicDNA,Formaldehyde fixed,...,1613007,39133,Active Motif,1,1088,face,Excision,ChIP-seq,H3K27AC,41
5,63,NA,14130,Sonication,26,None,86,Histology,genomicDNA,Formaldehyde fixed,...,1613007,39133,Active Motif,1,1089,face,Excision,ChIP-seq,H3K27AC,42


### Example: combine conjunction and disjunctions in filters
Similar to the prior example, the filters allow combining of conjunctive and disjunctive operators. Like the `and` operator, we also overload the bitwise-or "`|`" operator for logical-or.

In [12]:
path = dataset.link(sample).link(assay).filter(
    ((assay.sample_type == 'ChIP-seq') & (assay.selection == 'H3K27AC')) |
    ((assay.sample_type == 'Input') & (assay.selection == 'genomicDNA')))

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/isa:sample/isa:assay/((sample_type=ChIP-seq)&(selection=H3K27AC));((sample_type=Input)&(selection=genomicDNA))


In [13]:
path.entities().dataframe

,alignment_id,cell_count,dataset,fragmentation_method,id,isolation_protocol,library_id,markers,molecule_type,pretreatment,...,reagent_batch_number,reagent_catalog_number,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,tracks_id
0,42,NA,14068,Sonication,2,,62,histology,genomicDNA,Formaldehyde fixed,...,NA,NA,NA,4,1,medial nasal process,excision,Input,genomicDNA,22.0
1,45,NA,14068,Sonication,5,,65,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,1,medial nasal process,excision,ChIP-seq,H3K27AC,25.0
2,47,NA,14068,Sonication,7,,67,histology,genomicDNA,Formaldehyde fixed,...,NA,NA,NA,4,3,latero nasal process,excision,Input,genomicDNA,27.0
3,49,NA,14068,Sonication,9,,69,histology,genomicDNA,Formaldehyde fixed,...,NA,NA,NA,4,4,mandibular process,excision,Input,genomicDNA,29.0
4,52,NA,14068,Sonication,12,,72,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,4,mandibular process,excision,ChIP-seq,H3K27AC,32.0
5,54,NA,14068,Sonication,14,,74,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,3,latero nasal process,excision,ChIP-seq,H3K27AC,34.0
6,56,NA,14068,Sonication,16,,76,histology,genomicDNA,Formaldehyde fixed,...,NA,NA,NA,4,2,maxillary process,excision,Input,genomicDNA,36.0
7,59,NA,14068,Sonication,19,,79,histology,genomicDNA,Formaldehyde fixed,...,1613007.0,39133.0,Active Motif,1,2,maxillary process,excision,ChIP-seq,H3K27AC,39.0
8,61,NA,14130,Sonication,21,None,81,Histology,genomicDNA,Formaldehyde fixed,...,1613007,39133,Active Motif,1,1088,face,Excision,ChIP-seq,H3K27AC,41.0
9,61,NA,14130,Sonication,24,None,84,Histology,genomicDNA,Formaldehyde fixed,...,NA,NA,NA,4,1088,face,Excision,Input,genomicDNA,NaN


### Example: filtering at different levels of the path
Filtering a data path does not have to be done at the end of a path. In fact, the initial intention of the ERMrest URI was to mimick "RESTful" semantics where a RESTful "resource" is identified, then filtered, then a "sub-resource" is identified, and then filtered, and so on.

In [14]:
path = dataset.filter(dataset.release_date >= '2017-01-01') \
    .link(sample).filter(sample.species == 2) \
    .link(assay).filter(assay.selection == 'totalRNA')
    
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/release_date::geq::2017-01-01/isa:sample/species=2/isa:assay/selection=totalRNA


In [15]:
path.entities().dataframe

,alignment_id,cell_count,dataset,fragmentation_method,id,isolation_protocol,library_id,markers,molecule_type,pretreatment,...,reagent_batch_number,reagent_catalog_number,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,tracks_id
0,41,NA,14068,Fragmentation Buffer from Illumina,1,,61,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,1,medial nasal process,excision,RNA-seq,totalRNA,21
1,46,NA,14068,Fragmentation Buffer from Illumina,6,,66,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,3,latero nasal process,excision,RNA-seq,totalRNA,26
2,55,NA,14068,Fragmentation Buffer from Illumina,15,,75,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,2,maxillary process,excision,RNA-seq,totalRNA,35
3,60,NA,14068,Fragmentation Buffer from Illumina,20,,80,histology,mRNA,Trizol,...,,15032619.0,Illumina,5,4,mandibular process,excision,RNA-seq,totalRNA,40


## Linking Examples

### Example: explicit column links
Up until now, the examples have shown how to link entities via _implicit_ join predicates. That is, we knew there existed a foriegn key reference constraint between foreign keys of one entity and keys of another entity. We needed only to ask ERMrest to link the entities in order to get the joined set.

The problem with implicit links is that it become _ambiguous_ if there are more than one foreign key reference between tables. To support these situations data paths can specify the columns to use for the join predicate explicitly.

In [16]:
path = dataset.link(dataset.id==sample.dataset)

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/isa:dataset/(id)=(isa:sample:dataset)


**IMPORTANT** Not all entities are related by foreign key references. ERMrest does not allow arbitrary relational joins. Entity sets must be related by a foreign key reference in order to link them in a data path.

In [17]:
path.entities(limit=3).dataframe

,_keywords,anatomy,collection_date,dataset,gender,gene,genotype,id,litter,local_identifier,mutation,origin,phenotype,replicate,species,specimen,stage,strain,theiler_stage
0,Mus musculus wild type CD1 E16.5 Medial nasal ...,213,None,14068,None,None,18,1,,E11.5_MNP,None,,66,None,2,10,30,4,15
1,Mus musculus wild type CD1 E16.5 Maxillary pro...,207,None,14068,None,None,18,2,,E11.5_MX,None,,66,None,2,10,30,4,15
2,Mus musculus wild type CD1 E16.5 Lateral nasal...,167,None,14068,None,None,18,3,,E11.5_LNP,None,,66,None,2,10,30,4,15


### Example: explicit column links assigned to an alias
In earlier examples we have linked entity sets and used table aliases to reference entities at any depth in the path structure. We can combine aliases with explicit column links using the same argument of the "`link(..., as_=...)`" method. We can then use that alias to access attributes and include them in the fetched entity set.

First, we'll define an alias for the '`sample`' table.

In [18]:
S = sample.as_('S')

 Then we will assign it during the explicit column link. Finally, we will include its attributes with the fetched '`dataset'` entities.

In [19]:
path = assay.link(assay.sample==sample.id, as_=S) \
    .link(sample.dataset==dataset.id) \
    .attributes(dataset.accession, dataset.release_date, dataset.title, S.species, S.stage)

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/isa:assay/S:=(sample)=(isa:sample:id)/(dataset)=(isa:dataset:id)/accession,release_date,title,S:species,S:stage


In [20]:
path.entities().dataframe

,accession,release_date,species,stage,title
0,FB00000806.2,2017-03-30,2,30,ChIP-seq of multiple histone marks and RNA-seq...
1,FB00000834,2017-02-06,2,30,Fine Tuning of Craniofacial Morphology by Dist...
2,FB00000807.2,2017-04-12,3,109,ChIP-seq of multiple histone marks and RNA-seq...
3,FB00000902,2017-09-01,2,25,Transcriptome Atlases of the Craniofacial Sutures
4,FB00000903,2017-09-01,2,4,Transcriptome Atlases of the Craniofacial Sutures
5,FB00000904,2017-09-01,2,25,Transcriptome Atlases of the Craniofacial Sutures
6,FB00000891,2017-09-11,2,30,Temporal analysis of ectoderm and mesenchyme e...


### Example: links with "outer join" semantics
Up until now, the examples have shown "`link`s" with _inner join_ semantics. _Outer join_ semantics can be expressed as part of explicit column links, and _only_ when using explicit column links.

The link methods accepts a "`join_type`" parameter, i.e., "`.link(... join_type='left')`", which may be `'left'`, `'right'`, `'full'`, and defaults to `''` which indicates inner join type.

By '`right`' outer joining in the link from `'sample'` to `'dataset`', the following path gives us a reference to `'dataset'` entities that _may or may not_ have any samples with assays.

In [21]:
path = assay.link(assay.sample==sample.id, as_=S) \
    .link(sample.dataset==dataset.id, join_type='right') \
    .attributes(dataset.accession, dataset.title, S.species, S.stage)

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/isa:assay/S:=(sample)=(isa:sample:id)/right(dataset)=(isa:dataset:id)/accession,title,S:species,S:stage


The previous _inner_ join example only returned 6 entities.

In [22]:
entities = path.entities()
len(entities)

712

We can see above that we have a full set of datasets _whether or not_ they have any samples with assays. For further evidence, we can convert to a DataFrame and look at a slice of its entries. Note that the sample's 'species' and 'stage' attributes do not exist for some entities (i.e., `NaN`).

In [23]:
entities.dataframe[700:]

,accession,species,stage,title
700,FB00000886,NaN,NaN,Skull vault formation in individual WT AB Fish
701,FB00000887,NaN,NaN,Skull vault formation in individual WT AB Fish
702,FB00000888,NaN,NaN,Skull vault formation in individual WT AB Fish
703,FB00000889,NaN,NaN,Skull vault formation in individual WT AB Fish
704,FB00000890,NaN,NaN,Skull vault formation in individual WT AB Fish
705,FB00000807.2,3.0,109.0,ChIP-seq of multiple histone marks and RNA-seq...
706,FB00000867,NaN,NaN,Temporal analysis of ectoderm and mesenchyme e...
707,FB00000902,2.0,25.0,Transcriptome Atlases of the Craniofacial Sutures
708,FB00000903,2.0,4.0,Transcriptome Atlases of the Craniofacial Sutures
709,FB00000904,2.0,25.0,Transcriptome Atlases of the Craniofacial Sutures


## Faceting Examples
You may have noticed that in the examples above, the 'species' and 'stage' attributes are numeric. These are internal "primary keys" that are not meaningful and difficult to filter on. We may want to construct filters on our datasets based on these categories. This can be used for "faceted search" modes and can be useful even within the context of programmatic access to data in the catalog.

### Example: faceting on "related" tables
Let's say we want to find all of the samples in our catalog where their species are 'Mus musculus' and their age stage are 'E10.5'.

We need to extend our understanding of the data model with the following tables that are related to '`sample`'.
- `isa.sample.species -> vocabulary.species.id`: the sample table has a foreign key reference to the '`species`' table.
- `isa.sample.stage -> vocabulary.stage.id`: the sample table has a foreign key reference to the '`stage`' table.

We may say that `species` and `stage` are _related_ to the `sample` table in the sense that `sample` has a direct foreign key relationship from it to them.

For convenience, we will get local variables for the species and stage tables.

In [24]:
species = pb.vocabulary.species
stage = pb.vocabulary.stage

We can now use our sample alias ('`S`') or make a new one.

In [25]:
S = sample.as_('S')

First, let's link samples with species and filter on the term "Mus musculus" (i.e., "mouse").

In [26]:
path = S.link(species).filter(species.term == 'Mus musculus')
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/vocabulary:species/term=Mus%20musculus


Next, we need to **reset the context** of the ERMrest data path. As stated in the introduction to the example 4 notebook, you need additional background knowledge of ERMrest to understand its semantics. To summarize, resetting the context tells the data path to refer back to an earlier entity set (based on its "alias") and continue as if we are joining or filtering off of that referenced entity set.

In [27]:
path = path.context_reset(S)
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/vocabulary:species/term=Mus%20musculus/$S


In [28]:
path = path.link(stage).filter(stage.term == 'E10.5')
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/vocabulary:species/term=Mus%20musculus/$S/vocabulary:stage/term=E10.5


Now, at this point the path would give us all of the "age stage" entities, but we want the "sample" entities. To get the samples, we will again **reset the context** of the data path back to samples.

In [29]:
path = path.context_reset(S)
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/vocabulary:species/term=Mus%20musculus/$S/vocabulary:stage/term=E10.5/$S


In [30]:
path.entities().dataframe

,_keywords,anatomy,collection_date,dataset,gender,gene,genotype,id,litter,local_identifier,mutation,origin,phenotype,replicate,species,specimen,stage,strain,theiler_stage
0,None,164,2015-07-14,14139,None,None,18,1331,None,E10.5 FnP Ect,None,None,None,None,2,None,12,3,14
1,None,133,2015-07-14,14139,None,None,18,1332,None,E10.5 MxP Ect,None,None,None,None,2,None,12,3,14
2,None,139,2015-07-14,14139,None,None,18,1333,None,E10.5 MdP Ect,None,None,None,None,2,None,12,3,14
3,None,190,2015-07-14,14139,None,None,18,1334,None,E10.5 FnP Mes,None,None,None,None,2,None,12,3,14
4,None,122,2015-07-14,14139,None,None,18,1335,None,E10.5 MxP Mes,None,None,None,None,2,None,12,3,14
5,None,139,2015-07-14,14139,None,None,18,1336,None,E10.5 MdP Mes,None,None,None,None,2,None,12,3,14


### Example: faceting and projecting facets from related tables
Let's repeat the example above but instead of returning the numeric key values in the entity set, we will get the meaningful terms from the vocabulary. To do this we will need a couple more aliases so that we can reference the column values we want.

In [31]:
O = species.as_('O') # 'O' for organism
A = stage.as_('A') # 'A' for age stage

_Note_: Since the above tables use the same column names and we want to fetch the "`term`" for both age and species, we will need to use column renaming in the attributes method.

In [32]:
path = S.link(species, as_=O).filter(species.term == 'Mus musculus') \
    .context_reset(S) \
    .link(stage, as_=A).filter(stage.term == 'E10.5') \
    .context_reset(S) \
    .attributes(S.id, S.collection_date, species=O.term, age_stage=A.term)
    
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/S:=isa:sample/O:=vocabulary:species/term=Mus%20musculus/$S/A:=vocabulary:stage/term=E10.5/$S/S:id,S:collection_date,species:=O:term,age_stage:=A:term


In [33]:
path.entities().dataframe

,age_stage,collection_date,id,species
0,E10.5,2015-07-14,1331,Mus musculus
1,E10.5,2015-07-14,1332,Mus musculus
2,E10.5,2015-07-14,1333,Mus musculus
3,E10.5,2015-07-14,1334,Mus musculus
4,E10.5,2015-07-14,1335,Mus musculus
5,E10.5,2015-07-14,1336,Mus musculus
